In [1]:
from sympy import symbols, sin, cos, pi, Point, Line
from sympy.simplify import simplify

In [280]:
import math

class ArchemedianSpiral:
    a, b, θ, ϕ = symbols('a,b,θ, ϕ')
    y1 = (a+b*θ)*sin(θ)
    x1 = (a+b*θ)*cos(θ)
    y2 = (a+b*ϕ)*sin(ϕ)
    x2 = (a+b*ϕ)*cos(ϕ)

    g = (b*sin(θ) + (a + b*θ)*cos(θ))/(b*cos(θ) - (a+b*θ)*sin(θ))
    h = (b*sin(ϕ) + (a + b*ϕ)*cos(ϕ))/(b*cos(ϕ) - (a+b*ϕ)*sin(ϕ))

    p1 = Point(x1,y1)
    p2 = Point(x2,y2)
    l1 = Line(p1, slope=g)
    l2 = Line(p2, slope=h)
    #self.int_point = self.l1.intersection(self.l2)[0]
    # hard code this so that it takes less time to create the class
    int_point = Point2D(
        (a**3*sin(θ) - a**3*sin(ϕ) + a**2*b*θ*sin(θ) - 2*a**2*b*θ*sin(ϕ) + 2*a**2*b*ϕ*sin(θ) 
         - a**2*b*ϕ*sin(ϕ) - a**2*b*cos(θ) + a**2*b*cos(ϕ) - a*b**2*θ**2*sin(ϕ) + 2*a*b**2*θ*ϕ*sin(θ) 
         - 2*a*b**2*θ*ϕ*sin(ϕ) + 2*a*b**2*θ*cos(ϕ) + a*b**2*ϕ**2*sin(θ) - 2*a*b**2*ϕ*cos(θ) - b**3*θ**2*ϕ*sin(ϕ) 
         + b**3*θ**2*cos(ϕ) + b**3*θ*ϕ**2*sin(θ) - b**3*ϕ**2*cos(θ))
        /(a**2*sin(θ - ϕ) + a*b*θ*sin(θ - ϕ) + a*b*ϕ*sin(θ - ϕ) + b**2*θ*ϕ*sin(θ - ϕ) + b**2*θ*cos(θ - ϕ) - 
          b**2*ϕ*cos(θ - ϕ) + b**2*sin(θ - ϕ)), 
        (((a + b*θ)
          *(b*cos(θ) - (a + b*θ)*sin(θ))
          *(a**2*sin(θ - ϕ) + a*b*θ*sin(θ - ϕ) + a*b*ϕ*sin(θ - ϕ) + b**2*θ*ϕ*sin(θ - ϕ) + b**2*θ*cos(θ - ϕ) - b**2*ϕ*cos(θ - ϕ) + b**2*sin(θ - ϕ))
          *sin(θ)) 
         - ((b*sin(θ) + (a + b*θ)*cos(θ))
            *(-(b*sin(θ) + (a + b*θ)*cos(θ))
              *(b*cos(ϕ) - (a + b*ϕ)*sin(ϕ))
              *((a + b*θ)*cos(θ) - (a + b*ϕ)*cos(ϕ)) 
              + ((b*cos(θ) - (a + b*θ)*sin(θ))
                *(b*cos(ϕ) - (a + b*ϕ)*sin(ϕ))
                *((a + b*θ)*sin(θ) - (a + b*ϕ)*sin(ϕ)))
              + ((a + b*θ)*cos(θ) - (a + b*ϕ)*cos(ϕ))
              *(a**2*sin(θ - ϕ) 
                + a*b*θ*sin(θ - ϕ) 
                + a*b*ϕ*sin(θ - ϕ) 
                + b**2*θ*ϕ*sin(θ - ϕ) 
                + b**2*θ*cos(θ - ϕ) 
                - b**2*ϕ*cos(θ - ϕ) 
                + b**2*sin(θ - ϕ))
             ))
        )/((b*cos(θ) - (a + b*θ)*sin(θ))
           *(a**2*sin(θ - ϕ) 
             + a*b*θ*sin(θ - ϕ) 
             + a*b*ϕ*sin(θ - ϕ) 
             + b**2*θ*ϕ*sin(θ - ϕ) 
             + b**2*θ*cos(θ - ϕ) 
             - b**2*ϕ*cos(θ - ϕ) 
             + b**2*sin(θ - ϕ))
          )
    )
    
    def __init__(self, a=0, b=1):
        self.a = a
        self.b = b

    @property
    def initial_point(self):
        """ Because quadratic curves are assuming that they are absolute we need to provide the starting point.
        """
        return list(map(float, self.p1.subs({"a":self.a, 
                                             "b":self.b, 
                                             "θ":0
                                            }).args))
        
    def get_intersection_points(self, num_cycles, angle_change, θ0=0):
        """
        num_cycles: 
            how many times does the spiral go around (this will be rounded up to give a complete segment)
        angle_change:
            how large is the change in angle for each quadratic segment
        """
        num_degs = num_cycles*360
        num_angles = math.ceil(num_degs/angle_change)

        δθ = (pi/180)*angle_change
        for x in range(num_angles):
            loc_θ = θ0 + x*δθ
            yield {"int_point": self.int_point.subs({"a":self.a, 
                                                     "b":self.b, 
                                                     "θ":loc_θ, 
                                                     "ϕ":(loc_θ+δθ)}),
                   "loc_θ": loc_θ,
                   "δθ": δθ
                  }

    def get_quad_control_points(self, num_cycles, angle_change, θ0=0):
        
        for x in self.get_intersection_points(num_cycles, angle_change, θ0=θ0):
            yield (tuple(map(float, x['int_point'].args)), 
                   tuple(map(float, self.p2.subs({"a":self.a, 
                                                  "b":self.b, 
                                                  "θ":x['loc_θ'], 
                                                  "ϕ":(x['loc_θ']+x['δθ'])
                                                 }).args))
                  )

            
#     def _html_repr_(self):
#         center = 500
#         d = f'M{center},{center} {" ".join("Q"+",".join(map(str, (q+center for q in quad[0])))+" "+",".join(map(str, (q+center for q in quad[1]))) for quad in quads)}'
#         myid = "blah"

#         svg(path(id="blah",
#                  fill="none", 
#                  stroke="red",
#                  d=d
#                 ),
#             text(
#                 textPath("The quick brown fox jumps over the lazy dog. "*10,
#                          **{"xlink:href":"#blah"}),

#             ),
#             width="1000",
#             height="1000"
#         )

        
            

In [281]:
blah = ArchemedianSpiral()

In [284]:
blah.a = pi
blah.b = 5
quad_points = list(blah.get_quad_control_points(2, 45, 0))

In [278]:
import math

from uuid import uuid4

from vdom.svg import *

from sympy import symbols, sin, cos, pi, Point, Line, Point2D
from sympy.simplify import simplify


quad_points

[((3.823049811347723, 0.4281721601096985),
  (4.298939429691716, 1.1518973484962065)),
 ((4.80122716118188, 1.9157683578322082),
  (4.987948251644099, 2.879793265790644)),
 ((5.186046460391281, 3.9025572555546892),
  (4.998243305428162, 4.998243305428162)),
 ((4.7986697105687925, 6.162600761794931),
  (4.188790204786391, 7.255197456936871)),
 ((3.5410661631457754, 8.415592486693809),
  (2.507070699668214, 9.356515229329029)),
 ((1.4109410137818736, 10.353979325408632), (0.0, 10.995574287564276)),
 ((-1.4920210128837004, 11.674038628699755),
  (-3.184657375254218, 11.885303129147685)),
 ((-4.969679277861052, 12.108098626707898),
  (-6.806784082777885, 11.789695867522417)),
 ((-8.738811282725296, 11.454841378198655),
  (-10.55184697812612, 10.55184697812612)),
 ((-12.453458410390402, 9.604736847418554),
  (-14.05694507281519, 8.115781021773632)),
 ((-15.734508770020259, 6.558039196801587),
  (-16.942878928785, 4.539830726426225)),
 ((-18.204081110948344, 2.433382707633732), (-18.84955592

In [285]:

# b = 5 # number of units between cycles
# num_cycles = 3 # minimum number of cycles to be covered
# angle = 45 # number of degrees each quadratic curve will approximate

# blah = ArchemedianSpiral(b=b)
# quad_iter = blah.get_quad_control_points(num_cycles=num_cycles, angle=angle)
# quads = list(quad_iter)

quads = quad_points

font_size = 12 # in px, 14 matches notebook default

center = float(b*(num_cycles*pi*2)+font_size)

d = f'M{center + blah.initial_point[0]},{center+ blah.initial_point[1]} {" ".join("Q"+",".join(map(str, (q+center for q in quad[0])))+" "+",".join(map(str, (q+center for q in quad[1]))) for quad in quads)}'
myid = f"{uuid4()}".replace("-","")

svg(path(id=f"{myid}",
         fill="none", 
         stroke="red",
         d=d
        ),
    text(
        textPath("The quick brown fox jumps over the lazy dog. "*10,
                 **{"xlink:href":f"#{myid}"}),
        style={"fontSize": f"{font_size}px"},
    ),
    width=f"{center*2 + 1.5*font_size}",
    height=f"{center*2 + 1.5*font_size}"
)


<svg height="230.4955592153876" width="230.4955592153876"><path d="M109.3893722612836,106.2477796076938 Q111.83892796416352,107.78688084783911 111.24602291312196,111.24602291312196 Q110.55367428898964,115.2853412582257 106.2477796076938,117.24335389525808 Q101.19764794774143,119.53979222939036 95.69593262956768,116.79962658581992 Q89.36014408567421,113.64404526456265 87.39822368615503,106.2477796076938 Q85.18204182468816,97.8929708216898 90.14232895686972,90.14232895686972 Q95.65566884061371,81.52752062436608 106.2477796076938,79.54424205218055 Q117.86919864057379,77.36823465826869 127.90683393121583,84.58872528417177 Q138.80589223665334,92.42887349108561 140.80529879718154,106.2477796076938 Q142.9580351032947,121.1264248493763 133.4604376039138,133.4604376039138 Q123.30533052545043,146.6483222492281 106.2477796076938,148.659280431156 Q88.11485768226478,150.79701684384597 73.48151793877585,139.01404127661175 Q58.001747360474475,126.54950743076336 55.98229715025711,106.2477796076938 Q53.85497854723675,84.86164042339408 67.92791426607789,67.92791426607789 Q82.69891589691446,50.15421717237337 106.2477796076938,48.128315516282626 Q130.88673801692744,46.00863334143659 150.12124862200767,62.37431059337993 Q170.19027613680885,79.45003629345871 172.22122533307947,106.2477796076938" fill="none" id="0135722e126149db9306fc70fbabb28d" stroke="red"></path><text style="font-size: 12px"><textPath xlink:href="#0135722e126149db9306fc70fbabb28d">The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. </textPath></text></svg>

In [257]:
quad_points

[((45.44023365525932, 3.9621028973339087),
  (45.17575577501087, 7.965704617030667)),
 ((44.907978161163335, 12.019256900076694),
  (43.926240954306735, 15.987844210572677)),
 ((42.932474805412745, 20.005057473507765),
  (41.23839237559251, 23.808996938995747)),
 ((39.52391877014031, 27.658723313295695),
  (37.14599949016053, 31.169194471905133)),
 ((34.73995413709381, 34.72118754810357),
  (31.730132480907848, 37.81449937761216)),
 ((28.685317778351166, 40.943774934096595),
  (25.117993877991495, 43.50564158088528)),
 ((21.50989127159396, 46.096793375256745),
  (17.480188612866634, 48.02642350165889)),
 ((13.405182526773181, 49.9777472530449),
  (9.026460971162274, 51.191604001241416)),
 ((4.599312557538961, 52.418885505418814), (0.0, 52.853981633974485)),
 ((-4.649364758415325, 53.293812713363465),
  (-9.329534215199876, 52.91041778016443)),
 ((-14.059832815803633, 52.52291631622471),
  (-18.674064134701798, 51.30656953954061)),
 ((-23.336950803252183, 50.0773968393976),
  (-27.73598